<a href="https://colab.research.google.com/github/SytherAsh/CyberScan_CyberSecurity_ChatBot/blob/main/CyberScan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain langchain-community faiss-cpu transformers pypdf langchain_groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attemptin

In [15]:
# prompt: mount drivw

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from google.colab import userdata
groq=userdata.get("Groq")
gemma=userdata.get("Gemma")
huggingFace=userdata.get("HuggingFace")

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN']=huggingFace

In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

def load_documents(file_path):
    """Load PDF documents using PyPDFLoader."""
    loader = PyPDFLoader(file_path)
    return loader.load()

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks using RecursiveCharacterTextSplitter."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_documents(documents)

def create_embeddings():
    """Create HuggingFace embeddings."""
    return HuggingFaceBgeEmbeddings(
        model_name="sentence-transformers/all-MiniLM-l6-v2",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )

def create_vectorstore(final_documents, embeddings):
    """Create a FAISS vector store from the documents and embeddings."""
    return FAISS.from_documents(final_documents, embeddings)

def setup_llm():
    """Set up the Hugging Face model."""
    return HuggingFaceHub(
        repo_id="mistralai/Mistral-7B-v0.1",
        model_kwargs={"temperature": 0.5, "max_length": 1000}
    )

def create_prompt_template():
    """Create a prompt template for the RetrievalQA."""
    prompt_template = """
    "You are a knowledgeable assistant specialized in extracting information from PDF documents."
    "When answering questions, you must base your responses solely on the content of the PDF provided."
    "Do not include any information or context outside of this document."

    context: {context}
    Question: {question}
    Answer:
    """
    return PromptTemplate(template=prompt_template, input_variables=["context", "question"])

def setup_retrieval_qa(llm, retriever, prompt):
    """Set up the RetrievalQA chain."""
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt}
    )

def main(file_path):
    """Main function to execute the document processing and retrieval setup."""
    documents = load_documents(file_path)
    final_documents = split_documents(documents)
    embeddings = create_embeddings()
    vectorstore = create_vectorstore(final_documents, embeddings)

    # Create a retriever from the vector store
    retriever = vectorstore.as_retriever()

    llm = setup_llm()
    prompt = create_prompt_template()
    retrieval_qa = setup_retrieval_qa(llm, retriever, prompt)

    return retrieval_qa



In [8]:
file_path = "/content/drive/MyDrive/Colab Notebooks/CyberScan_Datahack/Cybersecurity Report Template.pdf"
retrieval_qa = main(file_path)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-7-26bf962180d2>:33: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  return HuggingFaceHub(


In [ ]:
# from google.colab import files

# # Save your model
# joblib.dump(retrievalQA, 'retrieverQA_model.joblib')

# # Download the model
# files.download('retrieverQA_model.joblib')

In [20]:
import re

def arrange(res):
  if "Answer:" in res:
      answer = res.split("Answer:")[1].strip()
  elif "answer:" in res:
      answer = res.split("answer:")[1].strip()
  else:
      # Fallback to the entire result if no split pattern is found
      answer = res
  return answer

def print_with_newline_on_period(text):
    modified_text = re.sub(r'\.', '.\n', text)
    return modified_text

In [21]:
# result=retrieval_qa.invoke({"query":"What measures are in place to ensure the confidentiality, integrity, and availability of data?"})
# result=result['result']

In [23]:
  import json

# Load the JSON file
with open('/content/drive/MyDrive/Colab Notebooks/CyberScan_Datahack/ques_ans.json', 'r') as file:
    data = json.load(file)

question = []
answer = []
domain=[]
# Access each domain's question and answer
for domain_name, details in data.items():
    question.append(details['question'])

    answer.append(print_with_newline_on_period(details['answer']))
    domain.append(domain_name)
    # print(f"Domain: {domain_name}")
    # print(f"Question: {question}")
    # print(f"Answer: {answer}\n")


In [36]:
answer_predicted=[]
for i in range(len(question)):
  query=question[i]
  result=retrieval_qa.invoke({"query":query})
  result=result['result']
  result=arrange(result)
  answer_predicted.append(result)

  # print("-"*50)
  # print(f"Domain: {domain[i]}")
  # print("-"*50)

  # print(f"Question: {query}")
  # print("-"*50)
  # print(f"Retrieved_Answer: {result}\n")
  # print("-"*50)

  # print(f"Acutal_Req: {answer[i]}\n")
  # print("-"*50)
  # print("-"*50)



Acutal_Req: To mitigate risks, we enforce a layered security approach using firewalls, intrusion detection/prevention systems (IDS/IPS), VPNs for secure remote access, and strong network segmentation.
 Regular vulnerability scans and penetration testing are conducted, and network traffic is monitored for anomalies.
 Compliance standards like NIST SP 800-53 and ISO 27001 require strict access controls and periodic monitoring to secure network infrastructure.


--------------------------------------------------
--------------------------------------------------
Acutal_Req: We enforce strong, multi-factor authentication (MFA), least-privilege access principles, and periodic reviews of access rights.
 Privileged Access Management (PAM) is used for sensitive accounts, and single sign-on (SSO) is implemented where possible.
 Compliance with NIST 800-63B and ISO 27001 Annex A.
9 ensures that authentication mechanisms and access controls are robust and secure.


-------------------------------

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Calculate similarity between actual response and predicted answer with context
def evaluate_domain_response(question, actual_answer, predicted_answer):
    """Evaluate the risk score based on the similarity between actual and predicted answers."""
    # Combine question with answers
    actual_combined = f"{question} {actual_answer}"
    predicted_combined = f"{question} {predicted_answer}"

    # Vectorize and calculate similarity
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([actual_combined, predicted_combined])
    similarity = cosine_similarity(vectors[0], vectors[1])

    # Interpret similarity into a risk score (0 - high risk, 1 - low risk)
    risk_score = 1 - similarity[0][0]  # Inverse of similarity for risk (0 is high risk, 1 is low risk)

    return risk_score, similarity[0][0]  # Return risk score and similarity score

# Evaluate responses for each domain
domain_scores = {}
for i in range(len(question)):
    # Calculate score with context
    risk_score, similarity_score = evaluate_domain_response(question[i], answer,answer_predicted[i])

    # Store the risk score in the domain_scores dictionary
    domain_scores[domain[i]] = {
        'risk_score': risk_score,
        'similarity_score': similarity_score
    }

# Identify the weakest domain (highest risk)
weakest_domain = min(domain_scores, key=lambda x: domain_scores[x]['risk_score'])
print(f"Domain with highest risk: {weakest_domain}")
print(f"Scores by domain: {domain_scores}")


Domain with highest risk: Data Security
Scores by domain: {'Network Security': {'risk_score': 0.5449396535747137, 'similarity_score': 0.45506034642528626}, 'Identity and Access Management (IAM)': {'risk_score': 0.6501004126688494, 'similarity_score': 0.3498995873311506}, 'Incident Response': {'risk_score': 0.5775198194227021, 'similarity_score': 0.42248018057729786}, 'Cloud Security': {'risk_score': 0.6072083673609427, 'similarity_score': 0.3927916326390572}, 'Endpoint Security': {'risk_score': 0.7965414090600451, 'similarity_score': 0.20345859093995494}, 'Data Security': {'risk_score': 0.46115311909821255, 'similarity_score': 0.5388468809017875}, 'Application Security': {'risk_score': 0.6418364713509379, 'similarity_score': 0.3581635286490621}, 'Compliance and Governance': {'risk_score': 0.5882867801412099, 'similarity_score': 0.4117132198587901}, 'Risk Management': {'risk_score': 0.6682166160458569, 'similarity_score': 0.3317833839541431}, 'Threat Intelligence': {'risk_score': 0.6376

In [38]:
def print_domain_scores(domain_scores):
    for domain, scores in domain_scores.items():
        risk_score = scores['risk_score']
        similarity_score = scores['similarity_score']
        print("-"*50)
        print(f"Domain: {domain}")
        print("-"*50)

        print(f" Risk Score: {risk_score:.2f}")

        print(f"Similarity Score: {similarity_score:.2f}")
        print("-"*50)
        print("-"*50)


# Identify the weakest domain (highest risk)
weakest_domain = min(domain_scores, key=lambda x: domain_scores[x]['risk_score'])
print(f"Domain with highest risk: {weakest_domain}")

# Print scores for each domain
print_domain_scores(domain_scores)


Domain with highest risk: Data Security
--------------------------------------------------
Domain: Network Security
--------------------------------------------------
 Risk Score: 0.54
Similarity Score: 0.46
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
Domain: Identity and Access Management (IAM)
--------------------------------------------------
 Risk Score: 0.65
Similarity Score: 0.35
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
Domain: Incident Response
--------------------------------------------------
 Risk Score: 0.58
Similarity Score: 0.42
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
Domain: Cloud Security
--------------------------------------------------
 Ris